In [9]:
import pandas as pd
import random
import sys
import numpy as np

sys.path.append("../scripts")
from scripts.prompt_gen import generate_prompt
from scripts.translate_summary_json import translate_summary

from qa_config import llama_system_qa_prompt
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import streamlit as st

import os
from dotenv import load_dotenv
load_dotenv()

os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = 'true'


In [11]:
df = pd.read_csv("../outputs/eval.csv")
df.head()

,idx,full_note,summary,questions,section_1,section_2,section_1_ground_truth_summary,section_2_ground_truth_summary,ground_truth_summary,prompt
0,162935,"0 A 42-year-old, G3P2 L0 presented to the o...",{'visit motivation': 'Routine antenatal check-...,[['Details (Optional) : Are there any specific...,Surgeries,Patient Medical History,\nSurgeries:A Termination of pregnancy surgery...,\nMedical History:The patient's drug usage is ...,Visit Motivation: The patient reported Routine...,"[Start Full note:]\nA 42-year-old, G3P2 L0 pre..."
1,83436,"0 A 73-year-old male patient, who we knew t...","{'visit motivation': 'uT3', 'admission': [{'re...",[['Severity (Optional) : Does the note indicat...,Diagnosis Tests,Admission,\nDiagnosis Tests:The Fiberoptic bronchoscopy ...,\nAdmission:The patient was admitted for Open ...,Visit Motivation: The patient reported uT3\nAd...,[Start Full note:]\nA 73-year-old male patient...
2,87502,0 A 3 years old boy presented with history ...,{'visit motivation': 'History of recurrent fev...,[['Details (Optional) : Are there any addition...,Symptoms,Patient Medical History,\nSymptoms:The patient reports Recurrent fever...,\nMedical History:The patient's physiological ...,Visit Motivation: The patient reported History...,[Start Full note:]\nA 3 years old boy presente...
3,55287,0 A 13-year-old girl complained of a three-...,{'visit motivation': 'Complained of lower back...,[['Drug Usage : Does the note mention any drug...,Patient Medical History,Medical Examinations,\nMedical History:The patient's physiological ...,\nMedical Examinations:Physical Examination ex...,Visit Motivation: The patient reported Complai...,[Start Full note:]\nA 13-year-old girl complai...
4,113035,0 Case 1 was a 63-year-old man who presente...,"{'visit motivation': 'Productive cough, fever,...",[['Name : What medication or treatment was adm...,Treatments,Symptoms,\nTreatments:The patient received Prednisone t...,\nSymptoms:The patient reports Productive coug...,Visit Motivation: The patient reported Product...,[Start Full note:]\nCase 1 was a 63-year-old m...


In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", llama_system_qa_prompt),
        ("user", "{full_note_and_questions} [/INST]"),
    ]
)

llm = Ollama(model="llama2", temperature=0)
output_parser = StrOutputParser()

In [13]:
chain = prompt | llm | output_parser

In [ ]:
for row in df.to_dict("records"):
    full_note_and_question = row["prompt"]
    response = chain.invoke({"full_note_and_questions": full_note_and_question})         
    break